In [ ]:
# To create interactive plots
import plotly.graph_objects as go
import json
import pandas as pd
import nltk
from os import listdir
from os.path import isfile, join
import seaborn as sns
from tqdm import tqdm
import glob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os
import string
from nltk.stem import WordNetLemmatizer
from pattern.en import tag
from nltk.corpus import wordnet as wn
import warnings
import scipy.sparse as sp 
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
from nltk.corpus import reuters
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import nltk.data
import math
import re
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
import numpy as np
import pandas as pd
import scipy as sc
stop_words = stopwords.words('english')
ideal_sent_length = 20.0
stemmer = SnowballStemmer("english")
import re
from urllib.request import urlopen
#import gensim 
import numpy as np 
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import re
import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim

import spacy
import joblib
from tqdm import tqdm
import seaborn as sb
from matplotlib import pyplot as plt

wnl = WordNetLemmatizer()

In [ ]:
path = r"C:\bur\CORD-19-research-challenge"
all_json = glob.glob(f'{path}/**/*.json', recursive=True)
len(all_json)


In [ ]:
metadata_path = f'{path}/metadata.csv'
# load metadata
metadata = pd.read_csv(metadata_path)
metadata.head()


In [ ]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            dic={'title':[]} 
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.metadataa = []
            self.metadata = content['metadata']
            meta=self.metadata
            self.abstract = []
            self.body_text = []
            
            
           
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
                
           
                tit=meta['title']
                dic['title'].append(tit)
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
            self.metadataa = tit
    def __repr__(self):
        return f'{self.metadataa}...'
first_row = FileReader(all_json[0])
print(first_row)

In [ ]:
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [],'title':[]}
for idx, entry in enumerate(all_json):
    if idx % (len(all_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
    dict_['title'].append(content.metadataa)
    
    
papers = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text','title'])
papers.head()

In [ ]:
#append all json files to the corresponding metadata
full_df = papers\
    .merge(metadata.rename(columns={'sha':'paper_id'}).drop(['abstract','title'], axis=1), 
           on='paper_id', how='left')

full_df.head()

In [ ]:
full_df.to_csv('papers.csv', index=False)

In [ ]:
import csv
import pandas as pd
papes=pd.read_csv('papers.csv')


In [ ]:
papes.head()

In [ ]:
print(papes.title[822])

In [ ]:
#for each column ,number of empty values
full_df.isnull().sum()

In [ ]:
import plotly.graph_objects as go
column=['title','abstract','body_text','publish_time','authors']
for col in column:
    if full_df[col].isna().sum():
    
        # Count the filled and empty values in the column
        tmp_dict = full_df[col].isna().value_counts().to_dict()
       

    else:
        tmp_dict = {False: len(full_df[col]), True: 0}
        
    print('The column "{}" has:\n\n{} filled and\n{} empty values.'.format(col, tmp_dict[False], tmp_dict[True]))


    # Count the unique values in the column
    nunique = full_df[col].nunique()
    #duplicated=full_df[col].duplicated
    dupl= full_df[full_df.duplicated([col])].__len__()
    

    print('\n{} unique values are in the column.'.format(nunique))
    print('\n{} duplicated values are in the column.'.format(dupl))
    colors = ['lightslategray',] * 6
    colors[1] = 'rgb(158,202,225)'
    colors[3]='rgb(158, 188, 225)'
    colors[2]='rgb(213, 229, 245)'
    colors[5]='rgb(122, 163, 240)'
    colors[4]='rgb(158, 217, 225)'
    colors[0]='rgb(90, 143, 242)'
    #colors[2]='rgb(55, 83, 109)'
    columns=['total','filled','empty','Unique','duplicated' ]

    fig = go.Bar(x=columns, y=[len(full_df[col]),tmp_dict[False],tmp_dict[True],nunique, dupl],#marker_color='lightsalmon'
                 marker_color=colors)
    
    layout = go.Layout( 
                       yaxis_title='Value Count', 
                       xaxis_title='{}'.format(col))

    graph = go.Figure([fig], layout)
    
    
    #graph.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',marker_line_width=1.5, opacity=0.6)
    graph.show()
    graph.show()


In [ ]:
#number of duplicated values 
full_df[full_df.duplicated(['abstract','body_text','title','paper_id'])].__len__()

In [ ]:
full_df.drop_duplicates(['title'], inplace=True)
full_df.dropna(subset=['body_text'], inplace=True)

In [ ]:
#full_df[full_df.duplicated(['body_text'])].__len__()

In [ ]:
len(full_df)

In [ ]:
#drop duplicated values
full_df=full_df.drop_duplicates(subset=['abstract','body_text','title','paper_id'])

In [ ]:
full_df[full_df.duplicated(['abstract','body_text','title','paper_id'])].__len__()

In [ ]:
len(full_df)

In [ ]:
covid_terms =['covid', 'coronavirus disease 19', 'sars cov 2', '2019 ncov', '2019ncov', '2019 n cov', '2019n cov',
              'ncov 2019', 'n cov 2019', 'coronavirus 2019', 'wuhan pneumonia', 'wuhan virus', 'wuhan coronavirus',
              'coronavirus 2', 'covid-19', 'SARS-CoV-2', '2019-nCov']
keywords = ['sars-cov', 'sars', 'coronavirus', 'ace2', 'coronaviruses', 'ncov', 'covid-19', 'wuhan', 'spike', 'sars-cov-2','corona']
covid_terms = [elem.lower() for elem in covid_terms]
covid_terms = re.compile('|'.join(covid_terms))
print(covid_terms)

In [ ]:
def checkYear(date):
    print(int(date[0:4]))
    return int(date[0:4])

def checkCovid(row, covid_terms):
    return bool(covid_terms.search(row['body_text'].lower()))

In [ ]:
full_df['is_covid'] =full_df.apply(checkCovid, axis=1, covid_terms=covid_terms)

In [ ]:
df_covid_only = full_df[full_df['is_covid']==True]
df_covid_only = df_covid_only.reset_index(drop=True)
len(df_covid_only)

In [ ]:
import semanticscholar as sch
from IPython.display import display, Latex, HTML, FileLink
from sentence_transformers import SentenceTransformer, models
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

In [ ]:
def preprocessing(text):
    # remove mail
    text = re.sub(r'[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}', 'MAIL', text)
    # remove doi
    text = re.sub(r'https\:\/\/doi\.org[^\s]+', 'DOI', text)
    # remove https
    text = re.sub(r'(\()?\s?http(s)?\:\/\/[^\)]+(\))?', '\g<1>LINK\g<3>', text)
    # remove single characters repeated at least 3 times for spacing error (e.g. s u m m a r y)
    text = re.sub(r'(\w\s+){3,}', ' ', text)
    # replace tags (e.g. [3] [4] [5]) with whitespace
    text = re.sub(r'(\[\d+\]\,?\s?){3,}(\.|\,)?', ' \g<2>', text)
    # replace tags (e.g. [3, 4, 5]) with whitespace
    text = re.sub(r'\[[\d\,\s]+\]', ' ',text)
     # replace tags (e.g. (NUM1) repeated at least 3 times with whitespace
    text = re.sub(r'(\(\d+\)\s){3,}', ' ',text)
    # replace '1.3' with '1,3' (we need it for split later)
    text = re.sub(r'(\d+)\.(\d+)', '\g<1>,\g<2>', text)
    # remove all full stops as abbreviations (e.g. i.e. cit. and so on)
    text = re.sub(r'\.(\s)?([^A-Z\s])', ' \g<1>\g<2>', text)
    # correctly spacing the tokens
    text = re.sub(r' {2,}', ' ', text)
    text = re.sub(r'\.{2,}', '.', text)
    text=re.sub(r"[<>()(,)|&©ø\[\]\'\";?~*!]", ' ', text) #remove <>()|&©ø"',;?~*!
    text=re.sub("(\\t)", ' ', text) #remove escape charecters
    text=re.sub("(\\r)", ' ', text) 
    text=re.sub("(\\n)", ' ', text)
    text= re.sub("(\s+)",' ',text) #remove multiple space
    # return lowercase text
    return text.lower()

In [ ]:
df_covid_only['preproc_body_text'] = df_covid_only['body_text'].apply(preprocessing)

In [ ]:
df_covid_only['body_text_parags'] = df_covid_only['preproc_body_text'].apply(splitParagraph)
df_covid_only.head()

In [ ]:
df_covid_only.to_csv('sabrina.csv', index=False)

In [ ]:
final_df=pd.DataFrame()
final_df['paper_id']=full_df['paper_id']
final_df['doi']=full_df['doi']
final_df['title']=full_df['title']
final_df['body_text']=full_df['body_text']
final_df.reset_index(drop=True, inplace=True)
final_df.head()

In [ ]:
final_df['body_text'][0]

In [ ]:
SUMMARY_LENGTH = 12  # number of sentences in final summary
stop_words = stopwords.words('english')


In [ ]:
class Summarizer():

    def penn_to_wn(self,tag):
        
        if tag.startswith('N'):
            return 'n'
 
        if tag.startswith('V'):
        
            return 'v'
 
        if tag.startswith('J'):
            return 'a'
 
        if tag.startswith('R'):
            return 'r'
        return None 
 

    def tagged_to_synset(self,word, tag):
        wn_tag = self.penn_to_wn(tag)
    
        if wn_tag is None:
            return None
 
        try:
        
            return wn.synsets(word, wn_tag)[0]
        except:
            return None
 
    def sentence_similarity(self,sentence1, sentence2):
       
        # Tokenize and tag
        sentence1 = pos_tag(word_tokenize(sentence1))
        
        sentence2 = pos_tag(word_tokenize(sentence2))
      
 
        # Get the synsets for the tagged words
        synsets1 = [self.tagged_to_synset(*tagged_word) for tagged_word in sentence1]
        
        synsets2 = [self.tagged_to_synset(*tagged_word) for tagged_word in sentence2]
       
 
    # Filter out the Nones
        synsets1 = [ss for ss in synsets1 if ss]
      
        synsets2 = [ss for ss in synsets2 if ss]
        
 
        score, count = 0, 0
        for synset in synsets1:
            max_sim = 0.0
            maxx=0
            for ss in synsets2:
                
                sim=wn.wup_similarity(synset, ss)
                if sim is not None and sim > max_sim:
                
                       max_sim = sim
                   
           
        
            if max_sim is not None and max_sim!=0:
                score += max_sim
                count += 1
 
    # Average the values
        if count!=0:
            score /= count
            return score   
    def __init__(self, article):
           
            self._articles = []
            #i=1
           
            for row in article:
             #   if i<=5:
                    title=row[1]
                    #print(title)
                    body=row[3].replace('\n', ' ')
                    #print(body)
                    paper_id=row[0]
                    #doi=row[0]
                    if title!=''and body!='':
                        self._articles.append((paper_id,title,body))
              #      i=i+1
            
            
    def valid_input(self, headline, article_text):
        return headline != '' and article_text != ''    
    def normalize_corpus(self,corpus, lemmatize=True):
        
        normalized_corpus = []    
        for text in corpus:
            if lemmatize:
                text = self.lemmatize_text(text)
            else:
                text = text.lower()
            text = self.remove_special_characters(text)
            text = self.remove_stopwords(text)
            normalized_corpus.append(text)
        return normalized_corpus
    def pos_tag_text(self,text):
        wnl = WordNetLemmatizer()
        tagged_text = tag(text)
        tagged_lower_text = [(word.lower(), self.penn_to_wn(pos_tag))
                             for word, pos_tag in
                             tagged_text]
        return tagged_lower_text
        
         
    def score(self,article,query):
        """ Assign each sentence in the document a score"""
        maxx=0
        maxxx=0
        Query=[]
        Query.append(query)
        headline = article[1]
        sentences = self.split_into_sentences(article[2])
        
        querry=self.remove_smart_quotes(query)
        sentencess=self.split_into_sentences(article[2])
        sentencess.append(querry)
        
        #queryy=self.split_into_sentences(query)
        norm_corpus =self.normalize_corpus(sentences, lemmatize=True)
        norm_corpuss=self.normalize_corpus(sentencess, lemmatize=True)
        
        norm_model_answer =  self.normalize_corpus(Query, lemmatize=True) 
        norm_model_answerquery =  self.normalize_corpus(Query, lemmatize=True) 
        
        vectorizer, corpus_features = self.build_feature_matrix(norm_corpus,feature_type='frequency')
        vectorizerq, query_features = self.build_feature_matrix(norm_corpuss,feature_type='frequency')
        # extract features from model_answer
        model_answer_features = vectorizer.transform(norm_model_answer)
        model_answer_featuresquery = vectorizerq.transform(norm_model_answerquery)
        
        doc_lengths = [len(doc.split()) for doc in norm_corpus]
        doc_lengthss = [len(doc.split()) for doc in norm_corpuss] 
        
        #query_lengths = [len(doc.split()) for doc in norm_query]  
        avg_dl = np.average(doc_lengths) 
        avg_qr = np.average(doc_lengthss)
        
        corpus_term_idfs = self.compute_corpus_term_idfs(corpus_features, norm_corpus)
        corpus_term_idfsquery = self.compute_corpus_term_idfs(query_features, norm_corpuss)
        
        for index, doc in enumerate(Query):
    
            doc_features = model_answer_features[index]
            #doc_featuress = model_answer_featuresquery[index]
            self.bm25_scores = self.compute_bm25_similarity(doc_features,corpus_features,doc_lengths,avg_dl,corpus_term_idfs,k1=1.5, b=0.75)         
            maxxx=max(self.bm25_scores)
            self.semantic_similarity_scores=[]
     
        for indexx, doc in enumerate(Query):

                doc_featuress = model_answer_featuresquery[indexx]
                self.bm25_scoresquery = self.compute_bm25_similarityqr(doc_featuress,query_features,doc_lengthss,avg_qr,corpus_term_idfsquery,k1=1.5, b=0.75)
            
                maxx=max(self.bm25_scoresquery)
        for i, s in enumerate(sentences):
                score1=self.sentence_similarity(s,query)
                score2=self.sentence_similarity(query,s)
                if score1 is not None and score2 is not None:
                    score=(score1+score2)/2
                    self.semantic_similarity_scores.append(score)
                elif score1 is not None and score2 is None:
                    self.semantic_similarity_scores.append(score1)
                elif score2 is not None and score1 is None:
                    self.semantic_similarity_scores.append(score2)
        doc_index=0
        sim_score=[]
        for score_tuple in zip(self.semantic_similarity_scores,self.bm25_scores):
            sim_score.append((score_tuple[0]+(score_tuple[1]/maxxx))/2)
        for tuple_ in zip(sentences,sim_score):
            s=tuple_[0]
            self._scores[s]=tuple_[1]
        
        
                    
              
    def generate_summaries(self,query):
        
                self.dict_ = {'task':[],'paper_id':[],'title':[],'summary': [],'score':[],'sentences':[]}
                jj=1
                ii=1
          
                #tasks=['what is the immune system response to 2019-ncov ?'
                   
                 #  ]
           
    
                #for query in tasks:
                for article in self._articles:
                    self._scores = Counter()
                    self.score(article,query)
                    highest_scoring = self._scores.most_common(SUMMARY_LENGTH)
                    #totalsentences = splitParagraph(article[2])
                    #summarylist=[]
                    #summr=[sent[0] for sent[0] in highest_scoring]
                   
                    #for sentence in totalsentences:
                     #   for sumsen in  summr:
                      #      if sentence==sumsen:
                       #         summarylist.append(sentence)        
                    # Appends highest scoring "representative" sentences, returns as a single summary paragraph.
                    summary=' '.join([sent[0] for sent in highest_scoring])
                    s=0
                    for scr in highest_scoring:
                        s=s+scr[1]
                    s=s/12    
                    
                    
                    ''' 
                    print('**task**')
                    print(query)
                    print(" **Title: **")
                    print(article[1])
                    print(highest_scoring)
                    print('**summary**')
                    print(summary)
                    print('____________________________________________________________________________________')
                    '''   
                    
                    self.dict_['sentences'].append(highest_scoring)
                    self.dict_['summary'].append(summary)
                    self.dict_['title'].append(article[1])
                    self.dict_['paper_id'].append(article[0])
                    self.dict_['task'].append(query)
                    self.dict_['score'].append(s)
                self.papers = pd.DataFrame(self.dict_, columns=['task','paper_id','title','summary','score','sentences'])
                return self.papers
       

    def remove_smart_quotes(self, text):
       
       
        #text=re.sub("([\(\[].*?[\)\]][\(\[].*?[\)\]]+)", ' ', text)
        text=re.sub("[\(\[].*?[\)\]]", '', text)
        
        try:
            url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', text)
            repl_url = url.group(3)
            text = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)',repl_url, text)
        except:
            pass #there might be emails with no url in them
        #text=re.sub("[\[]()*?[\]]", "", text)#remove in-text citation
        
        text=re.sub(r"[<>()(,)|&©ø\[\]\'\";?~*!]", ' ', text) #remove <>()|&©ø"',;?~*!
        text=re.sub("(\\t)", ' ', text) #remove escape charecters
        text=re.sub("(\\r)", ' ', text) 
        text=re.sub("(\\n)", ' ', text)
        text= re.sub("(\s+)",' ',text) #remove multiple space
        
        text = re.sub(r'[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}', 'MAIL', text)
    # remove doi
        text = re.sub(r'https\:\/\/doi\.org[^\s]+', 'DOI', text)
    # remove https
        text = re.sub(r'(\()?\s?http(s)?\:\/\/[^\)]+(\))?', '\g<1>LINK\g<3>', text)
    # remove single characters repeated at least 3 times for spacing error (e.g. s u m m a r y)
        text = re.sub(r'(\w\s+){3,}', ' ', text)
    # replace tags (e.g. [3] [4] [5]) with whitespace
        text = re.sub(r'(\[\d+\]\,?\s?){3,}(\.|\,)?', ' \g<2>', text)
    # replace tags (e.g. [3, 4, 5]) with whitespace
        text = re.sub(r'\[[\d\,\s]+\]', ' ',text)
     # replace tags (e.g. (NUM1) repeated at least 3 times with whitespace
        text = re.sub(r'(\(\d+\)\s){3,}', ' ',text)
    # replace '1.3' with '1,3' (we need it for split later)
        text = re.sub(r'(\d+)\.(\d+)', '\g<1>,\g<2>', text)
    # remove all full stops as abbreviations (e.g. i.e. cit. and so on)
        text = re.sub(r'\.(\s)?([^A-Z\s])', ' \g<1>\g<2>', text)
    # correctly spacing the tokens
        text = re.sub(r' {2,}', ' ', text)
        text = re.sub(r'\.{2,}', '.', text)
        text=re.sub(r"[<>()(,)|&©ø\[\]\'\";?~*!]", ' ', text) #remove <>()|&©ø"',;?~*!
        text=re.sub("(\\t)", ' ', text) #remove escape charecters
        text=re.sub("(\\r)", ' ', text) 
        text=re.sub("(\\n)", ' ', text)
        text= re.sub("(\s+)",' ',text) #remove multiple space
        text=re.sub("doi", ' ',text)
        text=re.sub("bioRxiv", ' ',text)
        text=re.sub("author", ' ',text)
        text=re.sub("authors", ' ',text)
        text=re.sub("authors", ' ',text)
        text=re.sub("All rights reserved", ' ',text)
        text=re.sub("preprint", ' ',text)
    # return lowercase text
        return text.lower()
        
        
        
      


    def split_into_sentences(self, text):
        tok = nltk.data.load('tokenizers/punkt/english.pickle')
        sentences = tok.tokenize(self.remove_smart_quotes(text))
        sentences = [sent.replace('\n', '') for sent in sentences if len(sent) > 20]   
        words=['author','authors''permissions','doi']
                                  
       
        
                 

        return sentences
    def lemmatize_text(self,text):
    
        pos_tagged_text = self.pos_tag_text(text)
        lemmatized_tokens = [wnl.lemmatize(word, pos_tag) if pos_tag
                             else word                     
                             for word, pos_tag in pos_tagged_text]
        lemmatized_text = ' '.join(lemmatized_tokens)
        return lemmatized_text

    def remove_special_characters(self,text):
        tokens = self.tokenize_text(text)
        pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
        filtered_tokens = filter(None, [pattern.sub(' ', token) for token in tokens])
        filtered_text = ' '.join(filtered_tokens)
        return filtered_text

    def remove_stopwords(self,text):
        stopword_list = nltk.corpus.stopwords.words('english')
        tokens = self.tokenize_text(text)
        filtered_tokens = [token for token in tokens if token not in stopword_list]
        filtered_text = ' '.join(filtered_tokens)    
        return filtered_text

    
  
    def build_feature_matrix(self,documents, feature_type='frequency',
                         ngram_range=(1, 1), min_df=0.0, max_df=1.0):

        feature_type = feature_type.lower().strip()  
    
        if feature_type == 'binary':
            vectorizer = CountVectorizer(binary=True, min_df=min_df,
                                     max_df=max_df, ngram_range=ngram_range)
        elif feature_type == 'frequency':
            vectorizer = CountVectorizer(binary=False, min_df=min_df,
                                     max_df=max_df, ngram_range=ngram_range)
        elif feature_type == 'tfidf':
            vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df, 
                                     ngram_range=ngram_range)
        else:
            raise Exception("Wrong feature type entered. Possible values: 'binary', 'frequency', 'tfidf'")

        feature_matrix = vectorizer.fit_transform(documents).astype(float)
    
        return vectorizer, feature_matrix

    def compute_corpus_term_idfs(self,corpus_features, norm_corpus):
    
        dfs = np.diff(sp.csc_matrix(corpus_features, copy=True).indptr)
        dfs = 1 + dfs # to smoothen idf later
        total_docs = 1 + len(norm_corpus)
        idfs = 1.0 + np.log(float(total_docs) / dfs)
        return idfs
    def compute_bm25_similarity(self,doc_features, corpus_features,
                            corpus_doc_lengths, avg_doc_length,
                            term_idfs, k1=1.5, b=0.75):
    # get corpus bag of words features
        corpus_features = corpus_features.toarray()
    # convert query document features to binary features
    # this is to keep a note of which terms exist per document
        doc_features = doc_features.toarray()[0]
        doc_features[doc_features >= 1] = 1
    
    # compute the document idf scores for present terms
        doc_idfs = doc_features * term_idfs
    # compute numerator expression in BM25 equation
        numerator_coeff = corpus_features * (k1 + 1)
        numerator = np.multiply(doc_idfs, numerator_coeff)
    # compute denominator expression in BM25 equation
        denominator_coeff =  k1 * (1 - b + 
                                (b * (corpus_doc_lengths / 
                                        avg_doc_length)))
        denominator_coeff = np.vstack(denominator_coeff)
        denominator = corpus_features + denominator_coeff
    # compute the BM25 score combining the above equations
        bm25_scores = np.sum(np.divide(numerator,
                                   denominator),
                         axis=1) 
    
        return bm25_scores
    def tokenize_text(self,text):
        tokens = nltk.word_tokenize(text) 
        tokens = [token.strip() for token in tokens]
        return tokens
    def compute_bm25_similarityqr(self,doc_features, corpus_features,
                            corpus_doc_lengths, avg_doc_length,
                            term_idfs, k1=1.5, b=0.75):
    # get corpus bag of words features
        corpus_features = corpus_features.toarray()
    # convert query document features to binary features
    # this is to keep a note of which terms exist per document
        doc_features = doc_features.toarray()[0]
        doc_features[doc_features >= 1] = 1
    
    # compute the document idf scores for present terms
        doc_idfs = doc_features * term_idfs
    # compute numerator expression in BM25 equation
        numerator_coeff = corpus_features * (k1 + 1)
        numerator = np.multiply(doc_idfs, numerator_coeff)
    # compute denominator expression in BM25 equation
        denominator_coeff =  k1 * (1 - b + 
                                (b * (corpus_doc_lengths / 
                                        avg_doc_length)))
        denominator_coeff = np.vstack(denominator_coeff)
        denominator = corpus_features + denominator_coeff
    # compute the BM25 score combining the above equations
        bm25_scores = np.sum(np.divide(numerator,
                                   denominator),
                         axis=1) 
    
        return bm25_scores

    



In [ ]:
import csv
import nltk
csv.field_size_limit(100000000)
results=pd.DataFrame()
f= open("sabrina.csv", encoding="utf-8-sig")
reader = csv.reader(f, delimiter=',')
next(reader)

summaries= Summarizer(reader)
results=summaries.generate_summaries('what is the immune system response to covid-19')
resultss = results.sort_values(by='score', ascending=False)
resultss.head(50)

            

In [ ]:
from rank_bm25 import BM25Okapi
import pandas as pd
data = pd.read_csv("sabrina2.csv")
print(len(data))

In [ ]:
from rank_bm25 import BM25Okapi
english_stopwords = list(set(stopwords.words('english')))

def strip_characters(text):
    t = re.sub('\(|\)|:|,|;|\.|’|”|“|\?|%|>|<', '', text)
    t = re.sub('/', ' ', t)
    t = t.replace("'",'')
    return t

def clean(text):
    t = text.lower()
    t = strip_characters(t)
    return t

def tokenize(text):
    words = nltk.word_tokenize(text)
    return list(set([word for word in words 
                     if len(word) > 1
                     and not word in english_stopwords
                     and not (word.isnumeric() and len(word) is not 4)
                     and (not word.isnumeric() or word.isalpha())] )
               )

def preprocess(text):
    t = clean(text)
    tokens = tokenize(t)
    return tokens

class SearchResults:
    
    def __init__(self, 
                 data: pd.DataFrame,
                 columns = None):
        self.results = data
        if columns:
            self.results = self.results[columns]
        print( "self.results")
        print( self.results)
            
    def __getitem__(self, item):
        return Paper(self.results.loc[item])
    
    def __len__(self):
        return len(self.results)
        
    def _repr_html_(self):
        return self.results._repr_html_()

SEARCH_DISPLAY_COLUMNS = ['paper_id', 'title', 'abstract', 'body_text']

class WordTokenIndex:
    
    def __init__(self, 
                 corpus: pd.DataFrame, 
                 columns=SEARCH_DISPLAY_COLUMNS):
        self.corpus = corpus
        print("self.corpus")
        print(self.corpus)
        raw_search_str = self.corpus.abstract.fillna('') + ' ' + self.corpus.title.fillna('') + ' ' + self.corpus.body_text.fillna('')
        print("raw_search_str")
        print(raw_search_str)
        self.index = raw_search_str.apply(preprocess).to_frame()
        print( "self.index")
        #print( self.index)
        print( self.index[0])
        self.index.columns = ['terms']
        print("self.index.columns")
        print(self.index.columns)
        self.index.index = self.corpus.index
        print("self.index.index")
        print(self.index.index)
        self.columns = columns
        print("self.columns")
        print(self.columns)
        return self.columns
    
    def search(self, search_string):
        search_terms = preprocess(search_string)
        print("search_terms" )
        print(search_terms )
        result_index = self.index.terms.apply(lambda terms: any(i in terms for i in search_terms))
        print("result_index")
        print(result_index )
        results = self.corpus[result_index].copy().reset_index().rename(columns={'index':'paper'})
        print("results")
        print(results)
        print("SearchResults(results, self.columns + ['paper'])")
        print(SearchResults(results, self.columns + ['paper']))
        return SearchResults(results, self.columns + ['paper'])

In [ ]:
class RankBM25Index(WordTokenIndex):
        
    def __init__(self, corpus: pd.DataFrame, columns=SEARCH_DISPLAY_COLUMNS):
        super().__init__(corpus, columns)
        #print("self.index.terms.tolist()")
        #print(self.index.terms.tolist())
        self.bm25 = BM25Okapi(self.index.terms.tolist())
        print("self.bm25")
        print(self.bm25)
        
    def search(self, search_string):
        search_terms = preprocess(search_string)
        doc_scores = self.bm25.get_scores(search_terms)
        print('doc_scores')
        print(doc_scores)
        ind = np.argsort(doc_scores)[::-1]
        print('ind')
        print(ind)
        results = self.corpus.iloc[ind][self.columns]
        print('results')
        print(results)
        results['Score'] = doc_scores[ind]
        print("results['Score']")
        print(results['Score'])
        results['orig_ind'] = ind
        results['word'] = search_string
        print("results['orig_ind']")
        print(results['orig_ind'])
        results = results[results.Score > 0]
        print("results")
        print(results)
        return SearchResults(results.reset_index(), self.columns + ['Score', 'orig_ind','word'])

In [ ]:
bm25_index = RankBM25Index(data)

In [ ]:

results = None
added = []
#for3 word in keywords:
    #print(word)
    #print("word_result")
word_result = bm25_index.search('range of incubation periods for the disease in humans').results
results = word_result
    
dc = results.sort_values(by='Score', ascending=False)

dc.reset_index(drop=True, inplace=True)
dc.head(12)

In [ ]:
dc.to_csv('index.csv', index=False)
import csv
import nltk
csv.field_size_limit(100000000)
results=pd.DataFrame()
f= open("index.csv", encoding="utf-8-sig")
reader = csv.reader(f, delimiter=',')
next(reader)

summaries= Summarizer(reader)
results=summaries.generate_summaries('range of incubation periods for the disease in humans')
resultss = results.sort_values(by='score', ascending=False)
resultss.head(50)


In [ ]:
#resultss.to_csv('results1.csv', index=False)
resultss.to_csv('nov.csv', index=False)

In [ ]:
resultss.shape

In [ ]:
print(resultss.summary[4])

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
word_cloud=WordCloud(width=400,height=300,background_color='white').generate(resultss.summary[732])
word_cloud.to_image()

In [ ]:
print(resultss.summary[732])

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
wordcloud = WordCloud().generate(resultss.summary[732])
plt.imshow(wordcloud, interpolation="bilinear")

In [ ]:
plt.imshow(wordcloud, interpolation="bilinear")

In [ ]:

word_cloud=WordCloud(width=400,height=300,background_color='white').generate(resultss.summary[732])
word_cloud.to_image()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
wordcloud = WordCloud().generate(resultss.summary[77])
plt.imshow(wordcloud, interpolation="bilinear")

In [ ]:

word_cloud=WordCloud(width=400,height=300,background_color='white').generate(resultss.summary[77])
word_cloud.to_image()

In [ ]:
print(resultss.summary[77])

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
wordcloud = WordCloud().generate(resultss.summary[355])
plt.imshow(wordcloud, interpolation="bilinear")

In [ ]:

word_cloud=WordCloud(width=800,height=300,background_color='white').generate(resultss.summary[335])
word_cloud.to_image()

In [ ]:
word_cloud=WordCloud(width=800,height=300,background_color='white').generate(resultss.summary[544])
word_cloud.to_image()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
wordcloud = WordCloud().generate(resultss.summary[394])
plt.imshow(wordcloud, interpolation="bilinear")

In [ ]:
word_cloud=WordCloud(width=800,height=300,background_color='white').generate(resultss.summary[438])
word_cloud.to_image()

In [ ]:
resultss.to_csv('wordnet+bm25.csv', index=False)

In [ ]:
csv2=pd.read_csv('w.csv')

In [ ]:
csv2.head()
csv2.shape

In [ ]:
import pandas as pd
fil=pd.read_csv('results1.csv')

In [ ]:
wordnetID=[]
wordnetScore=[]

for i in range(len(fil)):
    if i<50:
        wordnetID.append(fil.index[i])
        wordnetScore.append(fil.score[i])
print(wordnetID)
print(wordnetScore)
wordnetBM25_ID=[]
wordnetBM25_Score=[]

for i in range(len(csv2)):
     if i<50:
        wordnetBM25_ID.append(csv2.index[i])
        wordnetBM25_Score.append(csv2.score[i])
print(wordnetBM25_ID)
print(wordnetBM25_Score)

In [ ]:
plt.plot(wordnetID, wordnetScore,'g')
plt.plot(wordnetID, wordnetBM25_Score,'orange')
plt.xlabel('ID of document')
plt.ylabel('Score')

plt.show()